# Рекомендация тарифов

# Введение

Для выполнения этого задания, нам необходимо будет решить задачу классификации, поскольку в зависимости от признаков нам нужно определить наиболее подходящий из двух тарифов.

Сначала изучим данные, затем разделим их на три выборки:
- тренировочную
- валидационную
- тестовую

После этого, исследуем три модели:
- решающее дерево
- случайный лес
- логистическая регрессия

Для первых двух моделей рассмотрим различные гиперпараметры, чтобы найти наилучший результат.
После этого сравним результаты прогноза моделей с результатом константной модели, которая в любом случае проставит всем полученным данным результат тарифа ультра (1).

## Откройте и изучите файл

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

Загрузим данные и рассмотрим их. 

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')
print(data.info())
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Пропусков в данных нет, кроме того, данные уже предобработаны, следовательно проверять их не нужно.
В данных нам передано 3214 объектов, которые имеют 5 признаков:
- количество звонков
- количество использованных минут
- количество сообщений
- количество Мб интернета
- вид тарифа (1 - ультра, 0 - смарт).

За целевой признак примем вид тарифа, все остальные признаки будут использоваться для обучения.

## Разбейте данные на выборки

Разобъем данные на три выборки.

В первой выборке, которая составит 60%, будут данные для тренировки. Во второй (20%) - данные для валидации обученной модели. Третья выборка (тоже 20%) понадобится нам для финального тестирования лучшей модели.

In [3]:
target = data['is_ultra']
features = data.drop('is_ultra', axis=1)

features_train, features_test, target_train, target_test = train_test_split(
    features,
    target,
    test_size=0.2,
    random_state=12345
)

print(f'Размер тестовой выборки: {features_test.shape}')

features_train, features_valid, target_train, target_valid = train_test_split(
    features_train,
    target_train,
    test_size=0.25,
    random_state=12345
)

print(f'Размер валидационной выборки: {features_valid.shape}')
print(f'Размер тренировочной выборки: {features_train.shape}')


Размер тестовой выборки: (643, 4)
Размер валидационной выборки: (643, 4)
Размер тренировочной выборки: (1928, 4)


## Исследуйте модели

Как было сказано ранее, для прогноза проверим три модели:
- решающее дерево
- случайный лес
- логистическая регрессия

Во всех случаях, пороговым значением доли правильных ответов является 0.75, как указано в целях проекта.

Создадим словарь, куда впоследствии мы сохраним самые лучшие подобранные модели.

In [4]:
best_models={}

Для перебора гиперпараметров моделей будем использовать GridSearchCV.

### Решающее дерево

In [5]:
model_tree = DecisionTreeClassifier(random_state = 12345)

Рассмотрим входящие в модель гиперпараметры.

In [6]:
model_tree.get_params().keys()

dict_keys(['ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'random_state', 'splitter'])

Выбирая наилучшую модель типа "Решающее дерево", будем изменять следующие гиперпараметры:
- `max_depth` - максимальная глубина
- `criterion` - функция для измерения качества разбиения 
- `min_samples_split` - минимальное количество экземпляров, которое может содержаться в узле для его дальнейшего разбиения
- `min_samples_leaf` - минимальное количество экземпляров, которое может содержаться в листе

In [7]:
tree_parameters = {
    'criterion' : ['gini', 'entropy'],
    'max_depth' : range(1,11),
    'min_samples_leaf' : range(2,11,2),
    'min_samples_split' : range(2,11,2)
}

grid_tree = GridSearchCV(model_tree, tree_parameters, cv=3)

grid_tree.fit(features_train, target_train)
best_models['Решающее дерево'] = grid_tree.best_estimator_
print('Наилучшие параметры для модели Решающего дерева:')
for param_key in grid_tree.best_params_.keys():
    print(f'{param_key} : {grid_tree.best_params_[param_key]}')
print(f'Доля правильных ответов для данной модели: {grid_tree.best_score_}')

Наилучшие параметры для модели Решающего дерева:
criterion : gini
max_depth : 3
min_samples_leaf : 2
min_samples_split : 2
Доля правильных ответов для данной модели: 0.8101723327664812


Проверим данную модель на валидационной выборке.

In [8]:
predictions = grid_tree.best_estimator_.predict(features_valid)
result = accuracy_score(target_valid, predictions)
print(f'Доля правильных ответов для лучшей модели решающего дерева для валидационной выборки: {result}')

Доля правильных ответов для лучшей модели решающего дерева для валидационной выборки: 0.7651632970451011


В ходе исследования, лучшей моделью Решающего дерева оказалась модель со следующими параметрами:
- `criterion`: критерий Джини
- `max_depth`: 3
- `min_samples_split`: 2
- `min_samples_leaf` : 2

В этом случае доля правильных ответов модели составляет на валидационной выборке составила 0.76.

### Случайный лес

In [9]:
model_forest = RandomForestClassifier(random_state = 12345)

Рассмотрим входящие в модель гиперпараметры.

In [10]:
model_forest.get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

В случае с моделью "Случайный лес", для выбора наилучшей модели будем помимо тех же параметров одного дерева будем перебирать `n_estimators` - количество деревьев в модели.

In [11]:
forest_parameters = {
    'n_estimators' : range(10,51,10),
    'criterion' : ['gini', 'entropy'],
    'max_depth' : range(1,11),
    'min_samples_leaf' : range(2,11,2),
    'min_samples_split' : range(2,11,2)
}

grid_forest = GridSearchCV(model_forest, forest_parameters, cv=3)

grid_forest.fit(features_train, target_train)
best_models['Случайный лес'] = grid_forest.best_estimator_
print('Наилучшие параметры для модели Решающего дерева:')
for param_key in grid_forest.best_params_.keys():
    print(f'{param_key} : {grid_forest.best_params_[param_key]}')
print(f'Доля правильных ответов для данной модели: {grid_forest.best_score_}')

Наилучшие параметры для модели Решающего дерева:
criterion : entropy
max_depth : 9
min_samples_leaf : 8
min_samples_split : 2
n_estimators : 30
Доля правильных ответов для данной модели: 0.822624509656675


Проверим данную модель на валидационной выборке.

In [12]:
predictions = grid_forest.best_estimator_.predict(features_valid)
result = accuracy_score(target_valid, predictions)
print(f'Доля правильных ответов для лучшей модели случайного леса для валидационной выборки: {result}')

Доля правильных ответов для лучшей модели случайного леса для валидационной выборки: 0.7838258164852255


***Вывод***:

В ходе исследования, лучшей моделью Случайного леса оказалась модель со следующими параметрами:
- `criterion`: энтропийный критерий
- `max_depth`: 9
- `min_samples_split`: 8
- `min_samples_leaf` : 2
- `n_estimators` : 30

В этом случае доля правильных ответов модели составляет на валидационной выборке составила 0.78.

### Модель логистической регрессии

In [13]:
model_logreg = LogisticRegression(random_state=12345, solver = 'liblinear')

Рассмотрим входящие в модель гиперпараметры.

In [14]:
model_logreg.get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

Выбирая наилучшую модель логистической регрессии , будем изменять следующие гиперпараметры:
- `C` - степень регуляризации
- `penalty` - тип регуляризации
- `solver` - решатель

In [15]:
logreg_parameters = {
    'C' : np.logspace(-4, 4, 20),
    'penalty' : ['l1', 'l2']
}

grid_logreg = GridSearchCV(model_logreg, logreg_parameters, cv=3)

grid_logreg.fit(features_train, target_train)
best_models['Логистическая регрессия'] = grid_logreg.best_estimator_
print('Наилучшие параметры для модели Логистической регресии:')
for param_key in grid_logreg.best_params_.keys():
    print(f'{param_key} : {grid_logreg.best_params_[param_key]}')
print(f'Доля правильных ответов для данной модели: {grid_logreg.best_score_}')

Наилучшие параметры для модели Логистической регресии:
C : 4.281332398719396
penalty : l1
Доля правильных ответов для данной модели: 0.7515572286578522


Проверим данную модель на валидационной выборке.

In [16]:
predictions = grid_logreg.best_estimator_.predict(features_valid)
result = accuracy_score(target_valid, predictions)
print(f'Доля правильных ответов для лучшей модели логистической регрессии для валидационной выборки: {result}')

Доля правильных ответов для лучшей модели логистической регрессии для валидационной выборки: 0.7278382581648523


***Вывод***:

В ходе исследования, лучшей моделью Случайного леса оказалась модель со следующими параметрами:
- `С`: 4.281332398719396
- `penalty`: l1 (регрессия Лассо)

В этом случае доля правильных ответов модели составляет на валидационной выборке составила 0.72.

### Выбор наилучшей модели.

На основе проведенных испытаний, лучше всего проявила себя модель типа "Случайный лес" со следующими параметрами:
- `criterion`: энтропийный критерий
- `max_depth`: 9
- `min_samples_split`: 8
- `min_samples_leaf` : 2
- `n_estimators` : 30

В этом случае доля правильных ответов модели составляет на валидационной выборке составила 0.78.

Так же, пороговое значение преодолела модель решающего дерева со следующими параметрами:
- `criterion`: критерий Джини
- `max_depth`: 3
- `min_samples_split`: 2
- `min_samples_leaf` : 2

В этом случае доля правильных ответов модели составляет на валидационной выборке составила 0.76.

## Проверьте модель на тестовой выборке

Проверим все три модели на тестовой выборке.

In [20]:
for model in best_models:
    predictions = best_models[model].predict(features_test)
    result = accuracy_score(target_test, predictions)
    print(f'Для модели {model} на тестовой выборке доля правильных ответов составляет: {result}')

Для модели Решающее дерево на тестовой выборке доля правильных ответов составляет: 0.7869362363919129
Для модели Случайный лес на тестовой выборке доля правильных ответов составляет: 0.7947122861586314
Для модели Логистическая регрессия на тестовой выборке доля правильных ответов составляет: 0.7589424572317263


***Вывод***:

Как видно из результатов проверки, модель случайного леса дала лучший результат.
Стоит отметить, что модель логистической регрессии на тестовой выборке дала результат, превышающий пороговое значение.

##  Проверка модели на адекватность

Для проверки модель на адекватность, сравним ее результаты на тестовой выборке с результатами константной модели, которая для любых признаков предсказывает наиболее частый результат всей выборки.

In [21]:
model_dummy = DummyClassifier(strategy='most_frequent')
model_dummy.fit(features_train, target_train)
predictions = model_dummy.predict(features_test)
result = accuracy_score(target_test, predictions)
print(f'Для константной модели на тестовой выборке доля правильных ответов составляет: {result}')

Для константной модели на тестовой выборке доля правильных ответов составляет: 0.6951788491446346


***Вывод***:

Как мы видим, константная модель имеет долю правильных ответов 0.69. Следовательно, все рассмотренные выше модели можно считать адекватными.